In [1]:
import numpy as np
from datasets import dataset_dict
from datasets.colmap_utils import *


In [2]:

dataset = dataset_dict['phototourism']\
          ('/home/jupyter/data/IC_patient_000/segment_001/datasets/', 'train', img_downscale=3, use_cache=False, exp_name="IC_scale3_nerfw")



/opt/conda/envs/nerf_pl/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
rays_o, rays_d = dataset.all_rays[:10880, :3], dataset.all_rays[:10880, 3:6]
near, far = dataset.all_rays[:10880, 6:7], dataset.all_rays[:10880, 7:8]

start = (rays_o + near * rays_d).numpy()
end = (rays_o + far * rays_d).numpy()

### Blue points: scene geometry
### Red points: all cameras
### Purple point: an example camera
### Green frustum: the viewing frustum of the purple camera. It should cover the scene geometry. 

## You can move around using nbviewer!

In [8]:
import plotly.graph_objects as go

layout = go.Layout(
    yaxis=dict(
        range=[-0.01, 0.01],
    ),
    xaxis=dict(
        range=[-0.1, 0.1],

    ),
    autosize=False,
    width=800,
    height=600,
)
fig = go.Figure(layout=layout)

# Remove the axis labels
fig.update_yaxes(showticklabels=False)

# Remove the axis numbers
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

# # Constrain the x-axis to be from -1 to 1
# fig.update_xaxes(range=[-1, 1])

# # Constrain the y-axis to be from -1 to 1
# fig.update_yaxes(range=[-1, 1])
# #fig.update_zaxes(range=[-1,1])

z_in_range = dataset.xyz_world[:, 2]<5
skip = 5

# fig.add_trace(
#     go.Scatter3d(
#         x=dataset.xyz_world[z_in_range, 0][::skip],
#         y=dataset.xyz_world[z_in_range, 1][::skip],
#         z=dataset.xyz_world[z_in_range, 2][::skip],
#         mode='markers',
#         name='scene',
#         marker=dict(size=0.3)
#     )
# )

img_test = dataset.img_ids_test
img_train = dataset.img_ids_train
img_val = dataset.img_ids_val

test_poses = []
train_poses = []
val_poses = []
all_poses = []
for idx in img_test:
    test_poses.append(dataset.poses_dict[idx])
    all_poses.append(dataset.poses_dict[idx])

for idx in img_train:
    train_poses.append(dataset.poses_dict[idx])
    all_poses.append(dataset.poses_dict[idx])

for idx in img_val:
    val_poses.append(dataset.poses_dict[idx])
    all_poses.append(dataset.poses_dict[idx])

test_poses = np.array(test_poses)
train_poses = np.array(train_poses)
val_poses = np.array(val_poses)
all_poses = np.array(all_poses)


# fig.add_trace(
#     go.Scatter3d(
#         x=train_poses[:, 0, 3],
#         y=train_poses[:, 1, 3],
#         z=train_poses[:, 2, 3],
#         mode='markers',
#         name='cameras',
#         marker=dict(size=2)
#     )
# )

# fig.add_trace(
#     go.Scatter3d(
#         x=val_poses[:, 0, 3],
#         y=val_poses[:, 1, 3],
#         z=val_poses[:, 2, 3],
#         mode='markers',
#         name='cameras',
#         marker=dict(size=2)
#     )
# )

# fig.add_trace(
#     go.Scatter3d(
#         x=test_poses[:, 0, 3],
#         y=test_poses[:, 1, 3],
#         z=test_poses[:, 2, 3],
#         mode='markers',
#         name='cameras',
#         marker=dict(size=2)
#     )
# )
fig.add_trace(
    go.Scatter3d(
        x=all_poses[:, 0, 3],
        y=all_poses[:, 1, 3],
        z=all_poses[:, 2, 3],
        mode='markers',
        name='cameras',
        marker=dict(size=2)
    )
)

xlines = []
ylines = []
zlines = []
for i in [0, 127, -128, -1]:
    xlines += [start[i, 0], end[i, 0], None]
    ylines += [start[i, 1], end[i, 1], None]
    zlines += [start[i, 2], end[i, 2], None]
    
for p in [[0, 127], [127, -1], [-128, -1], [-128, 0]]:
    xlines += [start[p[0], 0], start[p[1], 0], None]
    ylines += [start[p[0], 1], start[p[1], 1], None]
    zlines += [start[p[0], 2], start[p[1], 2], None]
    xlines += [end[p[0], 0], end[p[1], 0], None]
    ylines += [end[p[0], 1], end[p[1], 1], None]
    zlines += [end[p[0], 2], end[p[1], 2], None]

# fig.add_trace(
#     go.Scatter3d(
#         x=xlines,
#         y=ylines,
#         z=zlines,
#         mode='lines',
#         name='frustum',
#         marker=dict(size=1)
#     )
# )

# pose = dataset.poses_dict[dataset.img_ids_train[0]]
# pose = dataset.poses_dict[dataset.img_ids_train[0]]

# img_test = dataset.img_ids_test
# img_train = dataset.img_ids_train
# img_val = dataset.img_ids_val
# for idx in img_test:
#     pose = dataset.poses_dict[idx]
#     fig.add_trace(
#         go.Scatter3d(
#             x=[pose[0, 3]],
#             y=[pose[1, 3]],
#             z=[pose[2, 3]],
#             mode='markers',
#             name='camera',
#             marker=dict(size=3, color='red')
#         )
#     )

fig.show()

In [5]:
root_dir = '/home/jupyter/data/BC_patient_007/segment_001/datasets/'
imdata = read_images_binary(os.path.join(root_dir, "sparse/0/images.bin"))

In [28]:
import numpy as np
from scipy.spatial import distance

poses_dict = dataset.poses_dict

def correct_scale(poses, root_dir, img_ids, w2c_mats):
  pts3d = read_points3d_binary(
      os.path.join(root_dir, "sparse/0/points3D.bin")
  )

  xyz_world = np.array([pts3d[p_id].xyz for p_id in pts3d])
  xyz_world_h = np.concatenate(
      [xyz_world, np.ones((len(xyz_world), 1))], -1
  )

  # Compute near and far bounds for each image individually
  nears, fars = {}, {}  # {id_: distance}
  for i, id_ in enumerate(img_ids):
      xyz_cam_i = (xyz_world_h @ w2c_mats[i].T)[
          :, :3
      ]  # xyz in the ith cam coordinate

      xyz_cam_i = xyz_cam_i[
          xyz_cam_i[:, 2] > 0
      ]  # filter out points that lie behind the cam

      nears[id_] = np.percentile(xyz_cam_i[:, 2], 0.1)
      fars[id_] = np.percentile(xyz_cam_i[:, 2], 99.9)

  max_far = np.fromiter(fars.values(), np.float32).max()
  scale_factor = max_far / 5  # so that the max far is scaled to 5
  poses[..., 3] /= scale_factor
  return poses

def remove_close_poses(imdata,poses_dict,root_dir, min_distance: 0.01, test_every_N: 20, plot=False):

  # create dict of points to ids and create points array
  point_to_id = {}
  points = []
  for id_, pose in poses_dict.items():
    xyz = pose[:,3]
    point_to_id[tuple(xyz)] = id_
    points.append(xyz)

  points = np.array(points)

  # compute distance of each point to all other points
  dist_matrix = distance.cdist(points, points, 'euclidean')

  dist_to_test_points = []
  test_points = []

  filtered_set = set()
  points_set = set([tuple(x) for x in points])

  # Iterate through all points
  for i in range(0, len(dist_matrix), test_every_N):
    for j in range(1, len(dist_matrix)):
      dist_to_points = dist_matrix[i]

      # Calculate distances from the current point to all points in test_points
      if len(test_points) > 0:
        dist_to_test_points = np.linalg.norm(test_points - points[i], axis=1)    

      # If the current point is too close to any point in the test set, skip it
      if np.any(dist_to_test_points) and np.min(dist_to_test_points) < min_distance:
        continue
      
      # Otherwise, add it to the test set
      test_points.append(points[i])

      # Find all points that are within min_distance of the current point
      close_point_indices = np.where((dist_to_points <= min_distance) & (dist_to_points > 0))[0]
      close_points = [points[i] for i in close_point_indices]

      # Add all close points to the filtered set
      for point in close_points:
        filtered_set.add(tuple(point))

      # Break the inner loop when we find a suitable point
      break
    
  # Remove the filtered and test points from the points set to get the train set
  test_set = set([tuple(x) for x in test_points])
  train_set = points_set - filtered_set - test_set

  # Get the filename for tsv generation
  train_images, test_images, filtered_images = [], [], []

  for point in train_set:
    id_ = point_to_id[point]
    train_images.append(imdata[id_].name)

  for point in test_set:
    id_ = point_to_id[point]
    test_images.append(imdata[id_].name)

  for point in filtered_set:
    id_ = point_to_id[point]
    filtered_images.append(imdata[id_].name)

  
  # Return sets with points if you want to plot the different sets
  if plot:
    return train_set, test_set, filtered_set
  else:
    return train_images, test_images, filtered_images

train_set, test_set, filt_set = remove_close_poses(imdata,poses_dict, root_dir,min_distance=0.02, test_every_N=20, plot=True)

In [29]:
train_poses = np.array([np.array(x) for x in train_set])
test_poses = np.array([np.array(x) for x in test_set])
filtered_poses = np.array([np.array(x) for x in filt_set])

In [30]:
import plotly.graph_objects as go

layout = go.Layout(
    yaxis=dict(
        range=[-0.01, 0.01],
    ),
    xaxis=dict(
        range=[-0.1, 0.1],

    ),
)
fig = go.Figure(layout=layout)


z_in_range = dataset.xyz_world[:, 2]<5
skip = 5


fig.add_trace(
    go.Scatter3d(
        x=train_poses[:, 0],
        y=train_poses[:, 1],
        z=train_poses[:, 2],
        mode='markers',
        name='train cameras',
        marker=dict(size=2)
    )
)

fig.add_trace(
    go.Scatter3d(
        x=filtered_poses[:, 0],
        y=filtered_poses[:, 1],
        z=filtered_poses[:, 2],
        mode='markers',
        name='filtered cameras',
        marker=dict(size=2, color='yellow')
    )
)

fig.add_trace(
    go.Scatter3d(
        x=test_poses[:, 0],
        y=test_poses[:, 1],
        z=test_poses[:, 2],
        mode='markers',
        name='cameras',
        marker=dict(size=2, color='red')
    )
)

xlines = []
ylines = []
zlines = []
for i in [0, 127, -128, -1]:
    xlines += [start[i, 0], end[i, 0], None]
    ylines += [start[i, 1], end[i, 1], None]
    zlines += [start[i, 2], end[i, 2], None]
    
for p in [[0, 127], [127, -1], [-128, -1], [-128, 0]]:
    xlines += [start[p[0], 0], start[p[1], 0], None]
    ylines += [start[p[0], 1], start[p[1], 1], None]
    zlines += [start[p[0], 2], start[p[1], 2], None]
    xlines += [end[p[0], 0], end[p[1], 0], None]
    ylines += [end[p[0], 1], end[p[1], 1], None]
    zlines += [end[p[0], 2], end[p[1], 2], None]



fig.show()